# Result-Dependent if/else

To branch based on the result from an electron, put the branch (if/else) logic inside another electron.

## Context

Often the output of one task is a value used to choose the execution path. In these cases, perform the if/else logic inside an electron. 

When a lattice is dispatched, the Covalent server executes the lattice in order to build the transport graph. The transport graph is then analyzed to parallelize electron execution on their assigned executors.

If the server encounters a branch decision based on the output of an electron, it cannot infer the structure on which the decision depends and is prevented from building the transport graph.

## Best Practice

Compute branching inside an electron. Electrons' execution is deferred during the graph build phase, so their output cannot be used to build the transport graph and analyze the execution for parallelization.

**Exception**: Sublattices are evaluated during lattice execution and their graph structure is "plugged in" to the larger transport graph. If a sublattice produces a static decision variable (does not produce it by executing an electron internally), then it is safe to branch on.

## Example

Contrast the two examples below.

### Example 1: Not Recommended

This example demonstrates a questionable approach: choosing the execution path in the lattice based on the output of `task_1`.

In [ ]:
import covalent as ct

# Technique 1:

@ct.electron
def task_1(x):
    return x * 2

@ct.electron
def task_2(x):
    return x ** 3

@ct.lattice
def workflow(a):
    
    res_1 = task_1(a)

    if res_1 == 10:
        final_res = task_2(res_1)
    else:
        final_res = res_1
    
    return final_res

### Example 2: Improved

The output of `task_1` is passed to the `task_2_new`, which executes the chosen path internally and returns the result.

In [ ]:
import covalent as ct

# Technique 2:

@ct.electron
def task_1(x):
    return x * 2

# Method (2):
@ct.electron
def task_2_new(x):
    if x == 10:
        return x ** 3
    else:
        return x

@ct.lattice
def workflow_2(a):
    res_1 = task_1(a)
    return task_2_new(res_1)

## See Also

[Result-Dependent Loops](./result_dependent_loop.ipynb)
